In [1]:
import os
import cv2
import numpy as np

def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            # Resize and normalize
            img_resized = cv2.resize(img, (16, 16)) / 255.0
            # Flatten
            img_flattened = img_resized.flatten()
            images.append(img_flattened)
    return np.array(images)

# Load images for class 0 and 5
class_0_images = load_images_from_folder("../data/svm/train/0")
class_5_images = load_images_from_folder("../data/svm/train/5")

class_0_images.shape, class_5_images.shape

((2380, 768), (2380, 768))

In [2]:
X = np.concatenate((class_0_images, class_5_images), axis=0)
y = np.concatenate((-1 * np.ones(class_0_images.shape[0]), np.ones(class_5_images.shape[0])), axis=0)

In [3]:
def gaussian_kernel(x1, x2, gamma=0.001):
    return np.exp(-gamma * np.linalg.norm(x1 - x2) ** 2)

In [4]:
from scipy.spatial.distance import cdist

# Compute pairwise squared distances
squared_dists = cdist(X, X, 'sqeuclidean')

# Compute Gaussian Kernel values
gamma = 0.001
K = np.exp(-gamma * squared_dists)

# Compute the P matrix
P = np.outer(y, y) * K

In [5]:
from cvxopt import matrix, solvers
from numpy import double

q = -np.ones((X.shape[0], 1))
G = np.vstack([-np.eye(X.shape[0]), np.eye(X.shape[0])])
h = np.hstack([np.zeros(X.shape[0]), np.ones(X.shape[0]) * 1.0])  # Considering C = 1.0
A = y.reshape(1, -1)
b = np.zeros(1)

P = matrix(P)
q = matrix(q)
G = matrix(G)
h = matrix(h)
A = matrix(A)
b = matrix(b)

sol = solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -3.0573e+03 -1.3944e+04  7e+04  3e+00  5e-13
 1: -2.0948e+03 -1.0001e+04  1e+04  2e-01  6e-13
 2: -2.2663e+03 -3.4451e+03  1e+03  2e-02  5e-13
 3: -2.6058e+03 -3.0009e+03  4e+02  5e-03  5e-13
 4: -2.7139e+03 -2.8783e+03  2e+02  2e-03  5e-13
 5: -2.7605e+03 -2.8255e+03  7e+01  6e-04  6e-13
 6: -2.7804e+03 -2.8029e+03  2e+01  2e-04  6e-13
 7: -2.7888e+03 -2.7935e+03  5e+00  2e-05  6e-13
 8: -2.7908e+03 -2.7913e+03  5e-01  2e-06  6e-13
 9: -2.7910e+03 -2.7910e+03  2e-02  6e-08  6e-13
10: -2.7910e+03 -2.7910e+03  7e-04  2e-09  6e-13
Optimal solution found.


In [6]:
class_0_val_images = load_images_from_folder("../data/svm/val/0")
class_5_val_images = load_images_from_folder("../data/svm/val/5")

X_val = np.concatenate((class_0_val_images, class_5_val_images), axis=0)
y_val = np.concatenate((-1 * np.ones(class_0_val_images.shape[0]), np.ones(class_5_val_images.shape[0])), axis=0)

In [7]:
sv_indices = ((alphas > 1e-5) & (alphas < 1 - 1e-5)).flatten()
support_vectors = X[sv_indices]
support_vector_labels = y[sv_indices]
alphas_sv = alphas[sv_indices]

# Calculate kernel matrix for support vectors
K_sv = np.exp(-0.001 * cdist(support_vectors, support_vectors, 'sqeuclidean'))

# Calculate b for each support vector
b_values = support_vector_labels - np.sum(alphas_sv * support_vector_labels * K_sv, axis=1)

# Average over all b values
b = np.mean(b_values)

In [8]:
b = -7.728114

In [9]:
sv_indices = ((alphas > 1e-5)).flatten()
support_vectors = X[sv_indices]
support_vector_labels = y[sv_indices]
alphas_sv = alphas[sv_indices]

def predict_gaussian(X):
    # Calculate Gaussian Kernel values between each point in X and the support vectors
    K = np.exp(-0.001 * cdist(X, support_vectors, 'sqeuclidean'))
    
    # Calculate predictions using the kernel values, alphas, support vector labels, and b
    predictions = np.dot(K, alphas_sv.flatten() * support_vector_labels.flatten()) + b
    
    return np.sign(predictions).flatten()

y_val_pred = predict_gaussian(X_val)
np.mean(y_val_pred == y_val)

0.5